<a href="https://colab.research.google.com/github/vence-andersen/M5-Forecasting-Accuracy/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# from google.colab import files
# files.upload()

In [3]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [4]:
# ! kaggle competitions download -c m5-forecasting-accuracy

In [5]:
# !unzip sales_train_evaluation.csv.zip
# !unzip sales_train_validation.csv.zip
# !unzip sample_submission.csv.zip
# !unzip sell_prices.csv.zip

In [6]:
# ! pip install downcast
# ! python -m pip install "dask[complete]"

In [7]:
import numpy as np
import pandas as pd
from downcast import reduce
from dask import dataframe as dd

In [8]:
# Using Pandas to reasd the files.

sales = pd.read_csv("sales_train_evaluation.csv")
sell_price = pd.read_csv("sell_prices.csv")
cal = pd.read_csv("calendar.csv")

In [9]:
for i in range(1941, 1941+29):
    sales['d_'+str(i)] = np.int8(0)

In [10]:
sales.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,2,4,0,0,0,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
sales.memory_usage().sum()/1024**2

453.5223560333252

In [12]:
sales = reduce(sales)

In [13]:
sales.memory_usage().sum()/1024**2

97.36645126342773

In [14]:
data = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
          var_name='day', value_name='demand').dropna()
print("The new shape is ",data.shape)
data.head(2)

The new shape is  (60034810, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [15]:
data.memory_usage().sum()/1024**2

1490.1800117492676

In [16]:
data = reduce(data)

In [17]:
data.memory_usage().sum()/1024**2

1146.751220703125

In [18]:
cal.fillna('No_Event', inplace=True)

In [19]:
cal = reduce(cal)

In [20]:
data = data.merge(cal, left_on='day', right_on='d')

In [21]:
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_Event,No_Event,No_Event,No_Event,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_Event,No_Event,No_Event,No_Event,0,0,0


In [22]:
data.shape

(60034810, 22)

In [23]:
data = reduce(data)

In [24]:
sell_price.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


In [25]:
sell_price.memory_usage().sum()/1024**2

208.77456665039062

In [26]:
sell_price = reduce(sell_price)

In [27]:
sell_price.memory_usage().sum()/1024**2

45.771300315856934

In [28]:
data = data.merge(sell_price,on=['store_id','item_id', 'wm_yr_wk'], how='left')

In [29]:
data.memory_usage().sum()/1024**2

2635.4417304992676

In [30]:
data = reduce(data)

In [31]:
data.memory_usage().sum()/1024**2

2635.4417304992676

In [32]:
print(f" There are {data['sell_price'].isna().sum()} missing values in sell_price column")

 There are 12299413 missing values in sell_price column


In [33]:
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_Event,No_Event,No_Event,No_Event,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,No_Event,No_Event,No_Event,No_Event,0,0,0,NaN


In [34]:
data['sell_price'].fillna(data.groupby('id')['sell_price'].transform('mean'), inplace=True)

In [35]:
print(f" There are {data['sell_price'].isna().sum()} missing values in sell_price column")

 There are 0 missing values in sell_price column


In [36]:
data.memory_usage().sum()/1024**2

2635.4417304992676

In [37]:
data = reduce(data)

In [38]:
data.memory_usage().sum()/1024**2

2635.4417304992676

In [39]:
print(f"There are {data.isna().sum().sum()} missing values in the data")

There are 0 missing values in the data


In [40]:
data['day'] = data['day'].str.split('_').str[1]

In [41]:
data.drop('d', axis=1, inplace=True)

In [42]:
data

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,No_Event,No_Event,No_Event,No_Event,0,0,0,8.281250
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,No_Event,No_Event,No_Event,No_Event,0,0,0,3.970703
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,No_Event,No_Event,No_Event,No_Event,0,0,0,2.970703
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,No_Event,No_Event,No_Event,No_Event,0,0,0,4.527344
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,Saturday,1,1,2011,No_Event,No_Event,No_Event,No_Event,0,0,0,2.941406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.980469
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.480469
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.980469
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.280273


In [43]:
data = reduce(data)

In [44]:
data.memory_usage().sum()/1024**2

2520.8412704467773

In [51]:
data.dtypes

id                    category
item_id               category
dept_id               category
cat_id                category
store_id              category
state_id              category
day                   category
demand                   int16
date            datetime64[ns]
wm_yr_wk                 int16
weekday               category
wday                      int8
month                     int8
year                     int16
event_name_1          category
event_type_1          category
event_name_2          category
event_type_2          category
snap_CA                   int8
snap_TX                   int8
snap_WI                   int8
sell_price             float16
dtype: object

In [52]:
data['id'].cat.codes

0           14370
1           14380
2           14390
3           14400
4           14410
            ...  
60034805    14329
60034806    14339
60034807    14349
60034808    14359
60034809    14369
Length: 60034810, dtype: int16